In [1]:
import os
import psycopg2
import requests
import json
import datetime
from pytz import timezone
import pandas as pd
import requests

In [2]:
futures = requests.get('https://ftx.com/api/futures').json()
df_futures = pd.DataFrame(futures['result'])
df_futures.set_index('name', inplace = True)
df_futures['symbol'] = df_futures.index

In [3]:
df_futures_db = df_futures.query('type != "move" & underlyingDescription == "Bitcoin"')

In [4]:
db_columns = ['symbol','expiry','last','bid','ask','index','mark','lowerBound','upperBound','marginPrice','change1h','change24h','changeBod','volumeUsd24h','volume','openInterest','openInterestUsd']

In [5]:
df_futures_db = df_futures_db[db_columns]

In [7]:
host="rdsnske.c7sczatjnphy.ap-northeast-1.rds.amazonaws.com"
dbname="bot"
user="rdsuser"
password="rds%5678"
insert_base = "insert into ftx_futures_btc values('{basetime}','{symbol}','{expiry}','{last}','{bid}','{ask}','{index}','{mark}','{lowerBound}','{upperBound}','{marginPrice}','{change1h}','{change24h}','{changeBod}','{volumeUsd24h}','{volume}','{openInterest}','{openInterestUsd}');"

In [10]:
# データをDBに登録
with psycopg2.connect(host=host, dbname=dbname, user=user, password=password) as conn:
    with conn.cursor() as cur:
        basetime = datetime.datetime.now()
        basetime = basetime.astimezone(timezone('UTC'))
        basetime = datetime.datetime.strftime(basetime, '%Y-%m-%dT%H:%M:00')
        for d in df_futures_db.itertuples():
            insert_sql = insert_base.format(basetime=basetime
                                            ,symbol=d.symbol
                                            ,expiry=d.expiry
                                            ,last=d.last
                                            ,bid=d.bid
                                            ,ask=d.ask
                                            ,index=d.index
                                            ,mark=d.mark
                                            ,lowerBound=d.lowerBound
                                            ,upperBound=d.upperBound
                                            ,marginPrice=d.marginPrice
                                            ,change1h=d.change1h
                                            ,change24h=d.change24h
                                            ,changeBod=d.changeBod
                                            ,volumeUsd24h=d.volumeUsd24h
                                            ,volume=d.volume
                                            ,openInterest=d.openInterest
                                            ,openInterestUsd=d.openInterestUsd)
            insert_sql = insert_sql.replace('None','2099-12-31T00:00:00')
            cur.execute(insert_sql)
    conn.commit()

In [23]:
for row in df_futures_db.itertuples():
    print(row.expiry)

None
2021-09-24T03:00:00+00:00
2021-12-31T03:00:00+00:00
2022-03-25T03:00:00+00:00


In [28]:
print(insert_sql)

insert into ftx_futures_btc values('2021-09-03T00:10:00','BTC-PERP','2099-12-31T00:00:00','50437.0','50436.0','50437.0','50346.579013327275','50440.0','47830.0','52936.0','50440.0','0.006465001197222444','0.062118340703305956','0.03407273770962319','4675678176.9091','95017.7758','42862.59','2161989039.6');


In [6]:
basetime = datetime.datetime.now()
basetime = basetime.astimezone(timezone('Asia/Tokyo'))